In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import json
from scipy.stats import linregress
from census import Census

# Census API Key
from config import api_key
c = Census(api_key, year=2017)

In [2]:
# Get census_data and save as CSV.file

census_data = c.acs5.get(("NAME", "B19013_001E", "B01003_001E", "B01002_001E",
                          "B19301_001E",
                          "B17001_002E"), {'for': 'zip code tabulation area:*'})

census_pd = pd.DataFrame(census_data)

census_pd = census_pd.rename(columns={"B01003_001E": "Population",
                                      "B01002_001E": "Median Age",
                                      "B19013_001E": "Household Income",
                                      "B19301_001E": "Per Capita Income",
                                      "B17001_002E": "Poverty Count",
                                      "NAME": "Name", "zip code tabulation area": "Zipcode"})

census_pd = census_pd[["Zipcode", "Population", "Median Age", "Household Income",
                       "Per Capita Income", "Poverty Count"]]

# Visualize
print(len(census_pd))
census_pd.to_csv("cencus_raw.csv")


33120


In [3]:
census_pd.describe()

,Population,Median Age,Household Income,Per Capita Income,Poverty Count
count,33120.000000,3.312000e+04,3.312000e+04,3.279500e+04,33120.000000
mean,9796.435085,-1.141300e+07,-4.350572e+07,-6.415586e+06,1424.893327
std,14510.547644,8.647927e+07,1.647643e+08,6.523051e+07,2760.345208
min,0.000000,-6.666667e+08,-6.666667e+08,-6.666667e+08,0.000000
25%,707.000000,3.650000e+01,3.900500e+04,2.115950e+04,76.000000
50%,2804.000000,4.170000e+01,5.063550e+04,2.612200e+04,327.000000
75%,13290.250000,4.700000e+01,6.456075e+04,3.263900e+04,1462.000000
max,119204.000000,8.950000e+01,2.500010e+05,2.981290e+05,38839.000000


In [4]:
census_file = pd.read_csv('cencus_raw.csv')
census_pd = pd.DataFrame(census_file)

census_pd.dtypes

Unnamed: 0             int64
Zipcode                int64
Population           float64
Median Age           float64
Household Income     float64
Per Capita Income    float64
Poverty Count        float64
dtype: object

In [5]:
# # Clean up the census_raw dataframe

# Replace <0 values with 0
census_pd[census_pd <= 0] = np.nan

# Replace
census_pd.dropna(axis=0, how='any',inplace=True)
census_pd

census_pd.to_csv("cleaned_census.csv",index=False)

In [6]:
census_cleanead_file = pd.read_csv('cleaned_census.csv')
census_df = pd.DataFrame(census_cleanead_file)
census_df["Zipcode"] = census_df['Zipcode'].astype(np.int64)
census_df.dtypes

Unnamed: 0           float64
Zipcode                int64
Population           float64
Median Age           float64
Household Income     float64
Per Capita Income    float64
Poverty Count        float64
dtype: object

In [7]:
census_df.head()

,Unnamed: 0,Zipcode,Population,Median Age,Household Income,Per Capita Income,Poverty Count
0,1.0,602,39209.0,40.9,16190.0,8978.0,20428.0
1,2.0,603,50135.0,40.4,16645.0,10897.0,25176.0
2,3.0,606,6304.0,42.8,13387.0,5960.0,4092.0
3,4.0,610,27590.0,41.4,18741.0,9266.0,12553.0
4,5.0,612,62566.0,41.4,17744.0,10283.0,29872.0


In [8]:
latitude_and_longitude = pd.read_csv("latitude_and_longitude.csv")
latitude_and_longitude_df = pd.DataFrame(latitude_and_longitude)
latitude_and_longitude_df.head()


,Zip,City,State,Latitude,Longitude,Timezone,Daylight savings time flag,geopoint
0,71937,Cove,AR,34.398483,-94.39398,-6,1,34.398483
1,72044,Edgemont,AR,35.624351,-92.16056,-6,1,35.624351
2,56171,Sherburn,MN,43.660847,-94.74357,-6,1,43.660847
3,49430,Lamont,MI,43.010337,-85.89754,-5,1,43.010337
4,52585,Richland,IA,41.194129,-91.98027,-6,1,41.194129


In [9]:
latitude_and_longitude_df = latitude_and_longitude_df.rename(columns={"Zip": "Zipcode"})
latitude_and_longitude_df.dtypes

Zipcode                         int64
City                           object
State                          object
Latitude                      float64
Longitude                     float64
Timezone                        int64
Daylight savings time flag      int64
geopoint                      float64
dtype: object

In [14]:
merged_df = pd.merge(census_df, latitude_and_longitude_df, how="inner", on='Zipcode')

merged_df = merged_df.rename(columns={"Median Age": "MedianAge", 
                          "Household Income": "HouseholdIncome",
                          "Per Capita Income": "PerCapitaIncome",
                          "Poverty Count": "PovertyCount",
                          "Daylight savings time flag": "DaylightSavingsTimeFlag"})
merged_df.head()

,Unnamed: 0,Zipcode,Population,MedianAge,HouseholdIncome,PerCapitaIncome,PovertyCount,City,State,Latitude,Longitude,Timezone,DaylightSavingsTimeFlag,geopoint
0,1.0,602,39209.0,40.9,16190.0,8978.0,20428.0,Aguada,PR,18.363285,-67.18024,-4,0,18.363285
1,2.0,603,50135.0,40.4,16645.0,10897.0,25176.0,Aguadilla,PR,18.448619,-67.13422,-4,0,18.448619
2,3.0,606,6304.0,42.8,13387.0,5960.0,4092.0,Maricao,PR,18.182151,-66.95880,-4,0,18.182151
3,4.0,610,27590.0,41.4,18741.0,9266.0,12553.0,Anasco,PR,18.288319,-67.13604,-4,0,18.288319
4,5.0,612,62566.0,41.4,17744.0,10283.0,29872.0,Arecibo,PR,18.449732,-66.69879,-4,0,18.449732


In [15]:
merged_df.to_csv("zip_merged_lat_lag.csv")

In [9]:
cleaned_merged = pd.read_csv("zip_merged_lat_lag.csv")
cleaned_merged_df = pd.DataFrame(cleaned_merged)
cleaned_merged_df = cleaned_merged_df.drop(columns=['Zipcode', 'Latitude', 'Longitude', 'Timezone', 'DaylightSavingsTimeFlag', 'geopoint'])
cleaned_merged_df.head()

,Population,MedianAge,HouseholdIncome,PerCapitaIncome,PovertyCount,City,State
0,39209,40.9,16190,8978,20428,Aguada,PR
1,50135,40.4,16645,10897,25176,Aguadilla,PR
2,6304,42.8,13387,5960,4092,Maricao,PR
3,27590,41.4,18741,9266,12553,Anasco,PR
4,62566,41.4,17744,10283,29872,Arecibo,PR


In [10]:
city_merged_df = cleaned_merged_df.groupby(['State','City']).agg({
    'Population':'sum',
    'MedianAge':'mean',
    'HouseholdIncome':'mean',
    'PerCapitaIncome':'mean',
    'PovertyCount':'sum'})

city_merged_df.head()

Population  MedianAge  HouseholdIncome  PerCapitaIncome  \
State City                                                                      
AK    Adak                   130       29.7          76250.0          21055.0   
      Akiachak               609       25.9          34444.0          11095.0   
      Akiak                  397       25.4          41250.0          10510.0   
      Akutan                 801       45.2          26750.0          26978.0   
      Alakanuk               735       20.7          33750.0           9521.0   
      Aleknagik              190       33.7          42083.0          23108.0   
      Allakaket              186       28.5          27250.0          15277.0   
      Ambler                 299       26.9          44500.0          15937.0   
      Anaktuvuk Pass         290       25.0          56667.0          16682.0   
      Anchor Point          2615       46.6          50913.0          29619.0   

                      PovertyCount  
State City                          
AK    Adak                      52  
      Akiachak                 209  
      Akiak                    139  
      Akutan                   152  
      Alakanuk                 323  
      Aleknagik                 25  
      Allakaket                 53  
      Ambler                    83  
      Anaktuvuk Pass            97  
      Anchor Point             271

In [12]:
city_merged_df.to_csv("city_merged.csv")